In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_sample_images

from utils import batch_plot
from conv import img2col, pooling, load_sample_filters, relu

In [ ]:
sample_images = np.asarray(load_sample_images().images)
sample_images = sample_images / sample_images.max()

# # for better plot
n = 420
sample_images = sample_images[:, :n, :n, :]

batch_plot(sample_images, with_border=False, imgsize=6)

# Pooling operations on images

<img src="./images/pooling_steps.gif">

In [ ]:
stride, padding = 2, 0
filter_height, filter_width = 2, 2
batch, height, width, channel = sample_images.shape

chunk_height, chunk_width = (height - filter_height + 2 * padding) // stride + 1, (
    width - filter_width + 2 * padding
) // stride + 1
max_pooling_sample_images = pooling(
    sample_images,
    kernel_size=(filter_height, filter_width),
    stride=stride,
    padding=padding,
    channel_first=False,
)
mean_pooling_sample_images = pooling(
    sample_images,
    kernel_size=(filter_height, filter_width),
    stride=stride,
    padding=padding,
    mode="mean",
    channel_first=False,
)

assert max_pooling_sample_images.shape == (batch, chunk_height, chunk_width, channel)
assert mean_pooling_sample_images.shape == (batch, chunk_height, chunk_width, channel)

for images, name in zip(
    [max_pooling_sample_images, mean_pooling_sample_images],
    ["max pooing", "mean pooling"],
):
    print(name)
    batch_plot(
        images,
        with_border=False,
        tight_layout=None,
        wspace=0.01,
        hspace=0.01,
        imgsize=6,
    )

# Conv + ReLU + Pooling

In [ ]:
filter_height = filter_width = 7
sample_filters = load_sample_filters(size=filter_height, sigma=1, channel=channel)
filters = np.asarray(list(sample_filters.values()))

stride, padding = 4, 10
num_filters = len(filters)
_, filter_height, filter_width, _ = filters.shape

batch, height, width, channel = sample_images.shape

chunk_height, chunk_width = (height - filter_height + 2 * padding) // stride + 1, (
    width - filter_width + 2 * padding
) // stride + 1

filtered_sample_images = (
    img2col(
        sample_images,
        kernel_size=(filter_height, filter_width),
        stride=stride,
        padding=padding,
        channel_first=False,
    )
    @ filters.reshape((num_filters, filter_height * filter_width * channel)).T
)
filtered_sample_images = filtered_sample_images.reshape((batch, chunk_height, chunk_width, num_filters)).transpose(
    (0, 3, 1, 2)
)

assert filtered_sample_images.shape == (batch, num_filters, chunk_height, chunk_width)

for filtered_sample_image in filtered_sample_images:
    batch_plot(
        filtered_sample_image,
        list(sample_filters.keys()),
        with_border=False,
        cmap=plt.cm.gray,
        tight_layout=None,
        wspace=0.1,
        hspace=0.1,
        imgsize=6,
    )

In [ ]:
activated_sample_images = relu(filtered_sample_images)

assert activated_sample_images.shape == (batch, num_filters, chunk_height, chunk_width)

for activated_sample_image in activated_sample_images:
    batch_plot(
        activated_sample_image,
        list(sample_filters.keys()),
        with_border=False,
        cmap=plt.cm.gray,
        tight_layout=None,
        wspace=0.1,
        hspace=0.1,
        imgsize=6,
    )

In [ ]:
stride, padding = 2, 0
filter_height, filter_width = 2, 2

pooling_sample_images = pooling(
    activated_sample_images.transpose((0, 2, 3, 1)),
    kernel_size=(filter_height, filter_width),
    stride=stride,
    padding=padding,
    channel_first=False,
)
pooling_sample_images = pooling_sample_images.transpose((0, 3, 1, 2))

for pooling_sample_image in pooling_sample_images:
    batch_plot(
        pooling_sample_image,
        list(sample_filters.keys()),
        with_border=False,
        cmap=plt.cm.gray,
        tight_layout=None,
        wspace=0.1,
        hspace=0.1,
        imgsize=6,
    )

- [An Intuitive Explanation of Convolutional Neural Networks](https://ujjwalkarn.me/2016/08/11/intuitive-explanation-convnets/)
- [A Beginner’s Guide To Understanding Convolutional Neural Networks](https://adeshpande3.github.io/A-Beginner's-Guide-To-Understanding-Convolutional-Neural-Networks/)